In [2]:
import re

In [3]:
import pandas as pd

In [4]:
import csv

In [5]:
from word2number import w2n

In [6]:
# read csv containing Dengue articles to a dataframe
df = pd.read_csv('TestDengueContent.csv')

In [10]:
from mysql.connector import MySQLConnection, Error

In [11]:
config = {
  'user': 'root',
  'password': '',
  'host': '127.0.0.1',
  'database': 'dengue_sri_lanka',
  'raise_on_warnings': True
}

In [12]:
import mysql.connector
from mysql.connector import Error
try:
#     mySQLconnection = mysql.connector.connect(host='localhost',
#                              database='dengue_sri_lanka',
#                              user='root',
#                              password='')
    conn = MySQLConnection(**config)
#     sql_select_Query = "select * from news_articles"
    select_new_articles = "select * from news_articles where isDataExtracted=0"

    cursor = conn .cursor()
    cursor.execute(select_new_articles)
    records = cursor.fetchall()
#     
    print("Total number of rows in news_articles is - ", cursor.rowcount)
    print ("Printing each row's column values i.e.  developer record")
    for row in records:
#        print("Id = ", row[0], )
#        print("Name = ", row[1])
#        print("JoiningDate  = ", row[2])
        print("Content  = ", row[3], "\n")
#     
    cursor.close()
   
except Error as e :
    print ("Error while connecting to MySQL", e)
finally:
    #closing database connection.
    if(conn .is_connected()):
        conn.close()
        print("MySQL connection is closed")


Total number of rows in news_articles is -  477
Printing each row's column values i.e.  developer record
Content  =  b'\xef\xbf\xbd\tHigher arrivals from Asia & Pacific and Europe\tChinese and Middle Eastern arrivals\xef\xbf\xbdslow down\xef\xbf\xbdTourist arrivals to Sri Lanka for the month of April rose 12.6 percent year-on-year (YoY) to 180, 429 amid higher arrivals from Asia & Pacific and Europe, the data released by Sri Lanka Tourism Development Authority (SLTDA) showed.Arrivals from Asia & Pacific, which include North East Asia, South East Asia, Oceania and South Asia grew 8.5 percent YoY to 82, 976, with South Asia accounting for the larger share.\xef\xbf\xbdArrivals from South Asian nations rose 4.7 percent YoY to 38, 299 with India leading the pack with 29, 860 tourists, up 13.4 percent YoY.Arrivals from China, which represents the North East Asia region, dropped 11.9 percent YoY to 17, 462. Japanese arrivals also grew by only 3.2 percent YoY to 3, 512.Arrivals from Oceania, w

Content  =  b'Eastern Province Chief Minister Sinavesathurai Chandrakanthan has returned back home and will restart his campaigning in the east this week after he was admitted to hospital with dengue, the TMVP said.\tThe Chief Minister was airlifted to a private hospital in Colombo from Batticaloa for treatment for dengue fever last week, party sources said.\tChandrakanthan had been treated at the Batticaloa Base Hospital for three days but had to be moved to Colombo after his condition worsened. (JN)' 

Content  =  b'By Indika Sri Aravinda\t\tThe Health Ministry has raised serious questions over the use of Bacillus Thuriengiensis (BTI) bacteria which is used to counter the larva of the dengue mosquito stating that it could have an adverse and a dangerous impact on the environment.\tOfficials at the Ministry told Daily Mirror online that although it was not \xef\xbf\xbdtoo keen\xef\xbf\xbd in importing the BTI, they had received orders from the \xef\xbf\xbdtop\xef\xbf\xbd to go ahead w

In [13]:
# Fill missing values
df = df.fillna("")

In [14]:
print(df.shape)

(479, 4)


In [15]:
# drops duplicate entries
df.drop_duplicates(subset=None, keep='first', inplace=True)

In [16]:
print(df.shape)

(478, 4)


In [17]:
regex_for_publication_date = re.compile(r"""(?:\d{4}-\d{2}-\d{2})|(?:\d{1,2}\s*(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|April|May|June|July|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s*\d{4})""",re.IGNORECASE)

In [18]:
regex_for_cases = re.compile(r'(\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d{1,3}){0,1})\s*(?:dengue\scases|dengue\sfever\scases|cases|suspected\sdengue\scases|suspected\scases|suspected\sdengue\sfever\scases|patients|dengue\spatients|from)', re.IGNORECASE)

In [19]:
regex_for_deaths = re.compile(r"""((?:\d+)|(?:[^\S]\d{1,3}(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)+)|
(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|
(?:(?:(?:twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety)\s*)(?:one|two|three|four|five|six|seven|eight|nine)?)))
\s*
(?:deaths|death|dead\b|(?:people\s*|patients\s*)?(?:have\s*|had\s*)?(?:died|dead\b)|dengue(?:\s|-)related\sdeath)
""", re.VERBOSE | re.IGNORECASE)

In [20]:
# regex to extract sentences having no. of cases/deaths
# TODO handle for "i.e" 
regex_for_sentences = re.compile(r"[^.]*?(?:\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)*\s*(?:dengue\scases|dengue\sfever\scases|cases|suspected\sdengue\scases|suspected\scases|suspected\sdengue\sfever\scases)|(?:(?:(?:\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)*)|(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|(?:(?:(?:twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety)\s*)(?:one|two|three|four|five|six|seven|eight|nine)?)))\s*(?:deaths|death|dead\b|(?:people\s*|patients\s*)?(?:have\s*|had\s*)?(?:died|dead\b)|patients|dengue\srelated\sdeath)))[^.]*\.")

In [21]:
# regex_for_years 20xx
regex_for_years = re.compile(r"""\b20(?:\d{2}){1}\b|this\syear""", re.IGNORECASE)

In [22]:
# regex for months
regex_for_months = re.compile(r'Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|April|May|June|July|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)')

In [23]:
# regex for "so far"
regex_for_so_far = re.compile(r'so\sfar')

In [44]:
regex_for_time_period_of = re.compile(r"""((?:\b20(?:\d{2}){1}\b|this\syear)|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|April|May|June|July|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember))|(?:so\sfar))""", re.IGNORECASE)

In [24]:
# regex for districts
regex_for_districts = re.compile(r'Jaffna|Kilinochchi|Mannar|Mullaitivu|Vavuniya|Puttalam|Kurunegala|Gampaha|Colombo|Kalutara|Anuradhapura|Polonnaruwa|Matale|Kandy|Nuwara\sEliya|Kegalle|Ratnapura|Trincomalee|Trinco|Batticaloa|Ampara|Badulla|Monaragala|Hambantota|Matara|Galle', re.IGNORECASE)

In [25]:
# regex for provinces
regex_for_provinces = re.compile(r'Western|Central|Eastern|North\sCentral|Northern|North\sWestern|Sabaragamuwa|Southern|Uva')

In [26]:
# regex for other places/locations
# regex_for_places = re.compile(r'Sri Lanka|island|country|countrywide|Meethotamulla|Negombo|Ratmalana|Dangolla|Elpitiya|Beliatte|Hatton|Marakolliya|Gampola|Kalubowila')

In [81]:
# regex for countries
regex_for_country = re.compile(r'Sri Lanka|island|country|countrywide')

In [47]:
regex_for_place_of = re.compile(r'((?:Jaffna|Kilinochchi|Mannar|Mullaitivu|Vavuniya|Puttalam|Kurunegala|Gampaha|Colombo|Kalutara|Anuradhapura|Polonnaruwa|Matale|Kandy|Nuwara\sEliya|Kegalle|Ratnapura|Trincomalee|Trinco|Batticaloa|Ampara|Badulla|Monaragala|Hambantota|Matara|Galle)|(?:Western|Central|Eastern|North\sCentral|Northern|North\sWestern|Sabaragamuwa|Southern|Uva)|(?:Sri Lanka|island|country|countrywide))', re.IGNORECASE)

In [27]:
def get_sentences(text):
    return regex_for_sentences.findall(text)

In [28]:
regex_for_dates = re.compile(r"""(?:\d{1,2}\s*(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|April|May|June|July|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s*\d{4})""",re.IGNORECASE)

In [29]:
def get_month_in_number(month):
    m = {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr':4,
        'may':5,
        'jun':6,
        'jul':7,
        'aug':8,
        'sep':9,
        'oct':10,
        'nov':11,
        'dec':12
        }
    s = month.strip()[:3].lower()

    try:
        out = m[s]
        return out
    except:
        raise ValueError('Not a month')
    

In [30]:
def get_published_date(text):
    dates = regex_for_publication_date.findall(text)
#     print(dates)
    for index,date in enumerate(dates):
        if(regex_for_dates.match(date)):
            day = re.search('\d{1,2}', date).group()
#             print(day)
#             print(re.search('\D{3,9}', date).group())
            month = get_month_in_number(re.search('\D{3,9}', date).group())
#             print(month)
            year = re.search('\d{4}', date).group()
#             print(year)
            date = year + "-" + str(month) + "-" + day
            dates[index] = date
    
    return dates

In [32]:
def get_cases(text):
    cases = regex_for_cases.findall(text)
    for index,case in enumerate(cases):
        case = case.replace(" ","")
        case = case.replace(",","")
        case = int(case)   
        cases[index] = case
    return cases

In [33]:
regex_for_digits = re.compile(r'(?:(?:\s)*\d+(?:(?:\s)\d+)*)')

In [52]:
def get_deaths(text):
    deaths = regex_for_deaths.findall(text)
#     print(deaths)
    for index,death in enumerate(deaths):
#         print(regex_for_digits.match(death))
        if(regex_for_digits.match(death)):
            death = death.replace(" ","")
            death = death.replace(",","")
#             print(death)
        death = w2n.word_to_num(death)   
        
        deaths[index] = death
    return deaths        

In [90]:
# ['2018']
def get_years(text,text_date):
    years = regex_for_years.findall(text)    
    for index,year in enumerate(years):
        if(year.lower() == "this year"):
            pub_date = get_published_date(text_date)[0]
            year = re.search('\d{4}',pub_date).group()
            years[index] = year
    return years
            

In [36]:
def get_months(text):
    return regex_for_months.findall(text)

In [38]:
def get_so_far(text):
    return regex_for_so_far.findall(text)

In [45]:
def get_time_period_of(text):
    return regex_for_time_period_of.findall(text)

In [39]:
def get_provinces(text):
    return regex_for_provinces.findall(text)

In [40]:
def get_districts(text):
    return regex_for_districts.findall(text)

In [82]:
def get_country(text):
    return regex_for_country.findall(text)

In [48]:
def get_place_of(text):
    return regex_for_place_of.findall(text)

In [119]:
# row = [published_date, cases, deaths, year, month, time, province, countrywide, district]
def get_info(published_date, cases, deaths, time_periods, places):
#     print(len(cases))
    lst = []
    for index,case in enumerate(cases):
        row = []
        if(len(published_date) != 0):    
            row.append(published_date[0])
        else:
            row.append("")
        row.append(case)
#         deaths
        if(len(deaths) > index):
            row.append(deaths[index])
        else:
#             row.append(None)
            row.append(0)
#         time_period
        if(len(time_periods) > index):
            if(len(get_years(time_periods[index], published_date[0])) != 0):
                row.append(get_years(time_periods[index], published_date[0])[0])
                row.append("")
                row.append("")
            elif(len(get_months(time_periods[index])) != 0):
                row.append("")
                row.append(get_months(time_periods[index])[0])
                row.append("")
            else:
                row.append("")
                row.append("")
                row.append(time_periods[index])
           
        else:
            row.append("")   
            row.append("")   
            row.append("")   

#        places
        if(len(places) > index):
        
            if(len(get_provinces(places[index])) != 0):
                row.append(get_provinces(places[index])[0])   
                row.append("")   
                row.append("") 
            elif(len(get_country(places[index])) != 0):
                row.append("")   
                row.append(get_country(places[index])[0])   
                row.append("")
            elif(len(get_districts(places[index])) != 0):
                row.append("")   
                row.append("")   
                row.append(get_districts(places[index])[0]) 
            else:
                row.append("")   
                row.append("")   
                row.append("")  
        
#             row.append(places[index])
        else:
            row.append("")   
            row.append("")   
            row.append("")  
        lst.append(row)
    return lst
    
    

In [42]:
articles=[]

In [131]:
conn = MySQLConnection(**config)
cursor = conn.cursor()

for index, row in df.iterrows():
#     print(regex_for_sentences.findall(row['Article_content']))
#     print("*")
    for sentence in regex_for_sentences.findall(row['Article_content']):
#         print(regex_for_sentences.findall(row['Article_content']))
#         print(get_deaths1(sentence))
#         print(get_published_date())
#         list = [1]
        lst = get_info(get_published_date(row['Date']),get_cases(sentence),get_deaths(sentence),get_time_period_of(sentence),get_place_of(sentence))
#         print(lst)
        for i,l1 in enumerate(lst):
#       row = [published_date, cases, deaths, year, month, time, province, countrywide, district]  
# '2018-02-09', 2000, 0, '2000', '', '', '', 'Sri Lanka', ''
#             query = "INSERT IGNORE INTO regex_output(publication_date, cases, deaths, year, month, time_duration, province, countrywide, article_id, district) " \
#                 "VALUES('2018-02-09', 2000, 0, '2000', '', '', '', 'Sri Lanka', 7, '')"
            print(l1)
#             print(l1[1])
#             print(l1[2])
#             print(l1[3])
#             print(l1[4])
#             print(l1[5])
#             print(l1[6])
#             print(l1[7])
#             print(l1[8])

            query = "INSERT INTO regex_output(publication_date, cases, deaths, year, month, time_duration, province, countrywide, article_id, district) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
            
            args = (l1[0], l1[1], l1[2], l1[3], l1[4], l1[5], l1[6], l1[7], 7, l1[8])
        
            try:
                cursor.execute(query, args)
                conn.commit()
            
            except Error as error:
                print(error)

#         print(get_published_date(row['Date']))
#         print(get_info(get_published_date(row['Date']),get_cases(sentence),get_deaths(sentence),get_time_period_of(sentence),get_place_of(sentence)))
#         print(get_cases(sentence))
#         print(get_deaths(sentence))
#         print(get_time_period_of(sentence))
#         print(get_place_of(sentence))
#         print()
        
# #     print(get_published_date(row['Date']))
# #         print(get_years(sentence,row['Date']))
# #         print(get_years1(sentence))
# #     print()
    

['2018-02-09', 2000, 0, '2000', '', '', '', 'Sri Lanka', '']
['2018-02-02', 184442, 0, '2017', '', '', '', 'island', '']
['2017-11-21', 150, 0, '', '', '', '', '', '']
['2017-11-21', 150, 0, '', '', '', '', '', '']
['2017-10-31', 162952, 0, '', '', '', '', '', '']
['2017-10-23', 80732, 0, '', 'January', '', '', '', '']
['2017-10-23', 18186, 215, '', '', '', '', '', 'Colombo']
['2017-08-30', 40625, 0, '', 'July', '', '', '', '']
['2017-08-30', 144052, 0, '', 'January', '', '', '', '']
['2017-08-25', 22186, 0, '', '', '', 'Western', '', '']
['2017-08-25', 17228, 0, '', '', '', '', '', 'Gampaha']
['2017-08-25', 5616, 0, '', '', '', '', '', 'Kalutara']
['2017-08-25', 3255, 0, '', '', 'so far', '', '', 'Jaffna']
['2017-08-25', 4135, 0, '2017', '', '', '', '', 'Batticaloa']
['2017-08-25', 4415, 0, '', '', 'so far', '', '', 'Trinco']
['2017-08-25', 5943, 0, '2017', '', '', '', '', 'Kurunegala']
['2017-08-25', 6081, 0, '', '', '', '', '', 'Ratnapura']
['2017-08-25', 100000, 0, '', '', '', '', 

In [ ]:
articles=[]
for index, row in df.iterrows():
    print("Index: "+str(index))
#     print("Publication date: "+get_published_date(row['Date']))
    sentences = get_sentences(row['Article_content'])
#     print(sentences)
    for sentence in sentences:
        print(sentence)
#         get_data(sentence)
        
        articles.append({
                'Index':str(index),
                'Publication_date':get_published_date(row['Date']),
                'Sentence':sentence.encode("utf-8"),
                'Numbers':get_cases1(sentence),
                'Cases':get_cases(sentence),
                'Deaths':get_deaths(sentence),
                'Years':get_years(sentence),
                'Months':get_months(sentence),
                'Dates': get_dates(sentence),
                'Time_duration':get_so_far(sentence),
                'Provinces':get_provinces(sentence),
                'Districts':get_districts(sentence),
                'Places':get_country(sentence),
            })
        
#     print()

In [ ]:
articles=[]
for row in records:
    sentences = get_sentences(row[3])
    for sentence in sentences:
        print(sentence)
        
        articles.append({
                'Index':str(index),
                'Publication_date':get_published_date(row[1]),
                'Sentence':sentence.encode("utf-8"),
                'Numbers':get_cases1(sentence),
                'Cases':get_cases(sentence),
                'Deaths':get_deaths(sentence),
                'Years':get_years(sentence),
                'Months':get_months(sentence),
                'Dates': get_dates(sentence),
                'Time_duration':get_so_far(sentence),
                'Provinces':get_provinces(sentence),
                'Districts':get_districts(sentence),
                'Places':get_country(sentence),
            })
        
#     print()

In [ ]:
articles=[]
conn = MySQLConnection(**config)
cursor = conn.cursor()

for row in records:
#     print(row[3].decode("utf-8"))
    sentences = get_sentences(row[3].decode("utf-8"))
    publication_date = get_published_date(row[1])

#     print(sentences)
    for sentence in sentences:
        print(sentence)
        
#         sentence = sentence.encode("utf-8")
#         numbers = get_cases1(sentence)
        cases_list = get_cases(sentence)
        deaths_list = get_deaths(sentence)
        year_list = get_years(sentence)
        month_list = get_months(sentence)
#         dates = get_dates(sentence)
        time_duration_list = get_so_far(sentence)
        province_list = get_provinces(sentence)
        district_list = get_districts(sentence)
        countrywide_list = get_country(sentence)
        
        for index,case in enumerate(cases_list):
            cases = 
            
        
        query = "INSERT IGNORE INTO regex_output(publication_date, cases, deaths, year, month, time_duration, province, countrywide, article_id, district) " \
            "VALUES(publication_date, cases, deaths, year, month, time_duration, province, countrywide, 1, district)"
        
        try:
#         db_config = read_db_config()
 
            cursor.execute(query)
            conn.commit()
            
        except Error as error:
            print(error)
        
cursor.close()
conn.close()

In [ ]:
articles=[]
conn = MySQLConnection(**config)
cursor = conn.cursor()

# for row in records:
#     print(row[3].decode("utf-8"))
#     sentences = get_sentences(row[3].decode("utf-8"))
#     print(sentences)
#     for sentence in sentences:
#         print(sentence)
        
#         publication_date = get_published_date(row[1])
# #         sentence = sentence.encode("utf-8")
# #         numbers = get_cases1(sentence)
#         cases = get_cases(sentence)
#         deaths = get_deaths(sentence)
#         year = get_years(sentence)
#         month = get_months(sentence)
# #         dates = get_dates(sentence)
#         time_duration = get_so_far(sentence)
#         province = get_provinces(sentence)
#         district = get_districts(sentence)
#         countrywide = get_country(sentence)
        
query = "INSERT IGNORE INTO regex_output(publication_date, cases, deaths, year, month, time_duration, province, countrywide, article_id, district) " \
            "VALUES('01/01/2018', 23455, 234, ['2019'], 'month', 'time', 'province', 'countrywide', 1, 'district')"
        
try:
#         db_config = read_db_config()
 
    cursor.execute(query)
    conn.commit()
            
except Error as error:
    print(error)
        
cursor.close()
conn.close()

In [ ]:
# TODO IMPORTANT: set the isDataExtracted column of each row to 1

In [ ]:
# write data in the 'article' list to a csv
with open('Regex_Output.csv',mode='w') as csv_file:
    fieldnames=['Index','Publication_date','Sentence','Numbers','Cases','Deaths','Years','Months','Dates','Time_duration','Provinces','Districts','Places']
    writer=csv.DictWriter(csv_file,fieldnames=fieldnames)

    writer.writeheader()
    for article in articles:
        writer.writerow({'Index':article['Index'],'Publication_date':article['Publication_date'],'Sentence':article['Sentence'],'Numbers':article['Numbers'],'Cases':article['Cases'],'Deaths':article['Deaths'],'Years':article['Years'],'Months':article['Months'],'Dates':article['Dates'],'Time_duration':article['Time_duration'],'Provinces':article['Provinces'],'Districts':article['Districts'],'Places':article['Places']})
